In [ ]:
import serial
from decimal import Decimal
import time
import threading
import os
#my_path = os.path.abspath(r'C:\Users\KuChris\Desktop\laser')
my_path = os.getcwd()

In [ ]:
##Reg
ITLA_NOERROR=0x00
ITLA_EXERROR=0x01
ITLA_AEERROR=0x02
ITLA_CPERROR=0x03
ITLA_NRERROR=0x04
ITLA_CSERROR=0x05
ITLA_ERROR_SERPORT=0x01
ITLA_ERROR_SERBAUD=0x02

REG_Nop=0x00
REG_Mfgr=0x02
REG_Model=0x03
REG_Serial=0x04
REG_Release=0x06
REG_Gencfg=0x08
REG_AeaEar=0x0B
REG_Iocap=0x0D
REG_Ear=0x10
REG_Dlconfig=0x14
REG_Dlstatus=0x15
REG_Channel=0x30
REG_Power=0x31
REG_Resena=0x32
REG_Grid=0x34
REG_Fcf1=0x35
REG_Fcf2=0x36
REG_LF1=0x40
REG_LF2=0x41
REG_Oop=0x42
REG_CTemp=0x43
REG_Opsl=0x50
REG_Opsh=0x51
REG_Lfl1=0x52
REG_Lfl2=0x53
REG_Lfh1=0x54
REG_Lfh2=0x55
REG_Currents=0x57
REG_Temps=0x58
REG_Ftf=0x62
REG_Mode=0x90
REG_PW=0xE0
REG_Csweepsena=0xE5
REG_Csweepamp=0xE4
REG_Cscanamp=0xE4
REG_Cscanon=0xE5
REG_Csweepon=0xE5
REG_Csweepoffset=0xE6
REG_Cscanoffset=0xE6
REG_Cscansled=0xF0
REG_Cscanf1=0xF1
REG_Cscanf2=0xF2
REG_CjumpTHz=0xEA
REG_CjumpGHz=0xEB
REG_CjumpSled=0xEC
REG_Cjumpon=0xED
REG_Cjumpoffset=0xE6

READ=0
WRITE=1
latestregister=0
tempport=0
raybin=0
queue=[]
maxrowticket=0

_error=ITLA_NOERROR
seriallock=0

##functions
def stripString(input):
    outp=''
    input=str(input)
    teller=0
    while teller<len(input) and ord(input[teller])>47:
        outp=outp+input[teller]
        teller=teller+1
    return(int(outp))

def checksum(byte0,byte1,byte2,byte3):
    bip8=(byte0&0x0f)^byte1^byte2^byte3
    bip4=((bip8&0xf0)>>4)^(bip8&0x0f)
    return bip4

def Send_command(sercon,byte0,byte1,byte2,byte3):
    sercon.write([byte0,byte1,byte2,byte3])

def Receive_response(sercon):
    global _error,queue
    reftime=time.perf_counter()
    while sercon.inWaiting()<4:
        if time.perf_counter()>reftime+0.25:
            _error=ITLA_NRERROR
            return(0xFF,0xFF,0xFF,0xFF)
        time.sleep(0.0001)
    try:
        byte0=ord(sercon.read(1))
        byte1=ord(sercon.read(1))
        byte2=ord(sercon.read(1))
        byte3=ord(sercon.read(1))
    except:
        print ('problem with serial communication. queue[0] =',queue)
        byte0=0xFF
        byte1=0xFF
        byte2=0xFF
        byte3=0xFF
    if checksum(byte0,byte1,byte2,byte3)==byte0>>4:
        _error=byte0&0x03
        return(byte0,byte1,byte2,byte3, hex(byte0),hex(byte1),hex(byte2),hex(byte3))
    else:
        _error=ITLA_CSERROR
        return(hex(byte0),hex(byte1),hex(byte2),hex(byte3))

def ITLA(sercon,register,data,rw):
    global latestregister
    lock=threading.Lock()
    lock.acquire()
    global queue
    global maxrowticket
    rowticket=maxrowticket+1
    maxrowticket=maxrowticket+1
    queue.append(rowticket)
    lock.release()
    while queue[0]!=rowticket:
        rowticket=rowticket
    if rw==0:
        byte2=int(data/256)
        byte3=int(data-byte2*256)
        latestregister=register
        Send_command(sercon,int(checksum(0,register,byte2,byte3))*16,register,byte2,byte3)
        test=Receive_response(sercon)
        b0=test[0]
        b1=test[1]
        b2=test[2]
        b3=test[3]
        if (b0&0x03)==0x02:
            test=AEA(sercon,b2*256+b3)
            lock.acquire()
            queue.pop(0)
            lock.release()
            return test
        lock.acquire()
        queue.pop(0)
        lock.release()
        return b2*256+b3
    else:
        byte2=int(data/256)
        byte3=int(data-byte2*256)
        Send_command(sercon,int(checksum(1,register,byte2,byte3))*16+1,register,byte2,byte3)
        test=Receive_response(sercon)
        lock.acquire()
        queue.pop(0)
        lock.release()
        return(test[2]*256+test[3])


def AEA(sercon,bytes):
    outp=''
    while bytes>0:
        Send_command(sercon,int(checksum(0,REG_AeaEar,0,0))*16,REG_AeaEar,0,0)
        test=Receive_response(sercon)
        outp=outp+chr(test[2])
        outp=outp+chr(test[3])
        bytes=bytes-2
    return outp

def ITLAConnect(port,baudrate):
    global CoBrite
    reftime=time.perf_counter()
    connected=False
    try:
        s = serial.Serial(port, baudrate , timeout=1, bytesize=8)
    except serial.SerialException:
        return(ITLA_ERROR_SERPORT)
    baudrate2=4800
    while baudrate2<115200:
        ITLA(s,REG_Nop,0,0)
        if ITLALastError()!=ITLA_NOERROR:
            #go to next baudrate
            if baudrate2==4800:baudrate2=9600
            elif baudrate2==9600: baudrate2=19200
            elif baudrate2==19200: baudrate2=38400
            elif baudrate2==38400:baudrate2=57600
            elif baudrate2==57600:baudrate2=115200
            s.close()
            s = serial.Serial(port,baudrate2 , timeout=1, bytesize=8)
        else:
            return(s)
    s.close()
    return(ITLA_ERROR_SERBAUD)

def ITLALastError():
    return(_error)    

In [ ]:
##Connection

#ITLAConnect("COM4",9600)
s = serial.Serial(port="COM4", baudrate=9600, bytesize=8, timeout=1, stopbits=serial.STOPBITS_ONE)

#if s.isOpen() == True:
    #print(s.name + ' is connected.')
#else:
    #print ('port is not connected.')

In [ ]:
ITLA(s,REG_Nop,0,0)

In [ ]:
##Setting
#Reset
ITLA(s,REG_Resena,0,1)

#ask for Lowest and highest Fre
lfl = (ITLA(s,REG_Lfl1,0,0) + ITLA(s,REG_Lfl2,0,0)*0.0001)
lfh = (ITLA(s,REG_Lfh1,0,0) + ITLA(s,REG_Lfh2,0,0)*0.0001)
print('lfl= '+str(lfl)+'Thz' +' ' 'lfh= ' + str(lfh)+'Thz')

Power = 1000
Grid = 1000
Channel = 1
#Fre = lfl
Fre = 191.3
finefre = 0

In [ ]:
#Nop
#ITLA(s,0x00,0,0)

#Temp
print('Temp = ' + str(ITLA(s,REG_CTemp,0,0)/100)+'C')


#Set PWR level
ITLA(s,REG_Power,Power,1)
#p = ITLA(s,REG_Oop,0,0)
print('Power = ' + str(Power*0.01) + 'dBm')

#Set Grid
ITLA(s,REG_Grid,Grid,1)
print('Grid = ' + str(Grid))

#Set Channel
Channel = 1
ITLA(s,REG_Channel,Channel,1)
print('Channel = ' + str(Channel))

#Set Frequency
Fre1 = int(Fre)
Fre2 = int(round((float(Fre) - Fre1) * 10000, 4))
ITLA(s,REG_Fcf1,Fre1,1)
ITLA(s,REG_Fcf2,Fre2,1)
fr = ITLA(s,REG_LF1,0,0)+ITLA(s,REG_LF2,0,0)*0.0001
print('Frequency = ' + str(fr) + 'THz')

#Set Fine tune Frequency
ITLA(s,REG_Ftf,finefre,1)
print('Fine tune Frequency = ' + str(ITLA(s,REG_Ftf,0,0)))

In [ ]:
ITLA(s,REG_Resena,8,1)

In [ ]:
ITLA(s,REG_Resena,0,1)

In [ ]:
##Auto
with open(os.path.join(my_path, r"data"+".txt" ), mode='a') as file:
    file.write('p;f;temp\n')

with open(os.path.join(my_path, r"data"+".txt" ), mode='a') as file:
    file.write('0;0;0\n')

#exec(open('testrealtime.py').read())

#Turn on the laser
ITLA(s,REG_Resena,8,1)
#time.sleep(5)

#nc=int((lfh-lfl)//(Grid/10000.0))
nc=int((196-191.3)//(Grid/10000.0))

#p = ITLA(s,REG_Oop,0,0)
#f = ITLA(s,REG_LF1,0,0) + ITLA(s,REG_LF2,0,0)*0.0001
#temp = ITLA(s,REG_CTemp,0,0)/100
#currents=[]
#for n in range(2,nc+2):
#    while True:
#        print(ITLA(s,REG_Oop,0,0))
#        open(os.path.join(my_path, r"data"+".txt"), mode='a').write(str(p)+';'+str(f)+';'+str(temp)+'\n')
#        time.sleep(1)
#        if ITLA(s,REG_Oop,0,0) == Power:
#            open(os.path.join(my_path, r"data"+".txt"), mode='a').write(str(p)+';'+str(f)+';'+str(temp)+'\n')
#            ITLA(s,REG_Channel,n,1)
#            break

##channel 1
print('Channel = ' + str(ITLA(s,REG_Channel,0,0)))
while ITLA(s,REG_Oop,0,0) < Power or ITLA(s,REG_Oop,0,0) > 10000:
    p = ITLA(s,REG_Oop,0,0)
    f = ITLA(s,REG_LF1,0,0) + ITLA(s,REG_LF2,0,0)*0.0001
    temp = ITLA(s,REG_CTemp,0,0)/100
    if ITLA(s,REG_Oop,0,0) > 10000:
        open(os.path.join(my_path, r"data"+".txt"), mode='a').write('0;{};{} \n'.format(f,temp))
        time.sleep(0.01)
    elif ITLA(s,REG_Oop,0,0) < Power:
        open(os.path.join(my_path, r"data"+".txt"), mode='a').write('{};{};{} \n'.format(p,f,temp))
        time.sleep(0.01)
    elif ITLA(s,REG_Oop,0,0) == Power:
        open(os.path.join(my_path, r"data"+".txt"), mode='a').write('{};{};{} \n'.format(p,f,temp))
        time.sleep(0.01)
        break

##channel 2-n
for n in range (2, nc+3):
    time.sleep(0.01)
    ITLA(s,REG_Channel,n,1)
    print('Channel = ' + str(ITLA(s,REG_Channel,0,0)))
    time.sleep(1)
    while ITLA(s,REG_Oop,0,0) < Power or ITLA(s,REG_Oop,0,0) > 10000:
        p = ITLA(s,REG_Oop,0,0)
        f = ITLA(s,REG_LF1,0,0) + ITLA(s,REG_LF2,0,0)*0.0001
        temp = ITLA(s,REG_CTemp,0,0)/100
        if ITLA(s,REG_Oop,0,0) > 10000:
            open(os.path.join(my_path, r"data"+".txt"), mode='a').write('0;{};{} \n'.format(f,temp))
            time.sleep(0.01)
        elif ITLA(s,REG_Oop,0,0) <= Power:
            open(os.path.join(my_path, r"data"+".txt"), mode='a').write('{};{};{} \n'.format(p,f,temp))
            time.sleep(0.01)
        #elif ITLA(s,REG_Oop,0,0) == Power:
        #    open(os.path.join(my_path, r"data"+".txt"), mode='a').write('{};{};{} \n'.format(p,f,temp))
        #    time.sleep(0.01)
        #    break

ITLA(s,REG_Resena,0,1)